Importando as bibliotecas

In [ ]:
import random
import requests
from collections import Counter
from math import log2

Inicializando o jogo da forca

In [ ]:
class JogoDeForca:
    def __init__(self):
        url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
        r = requests.get(url, allow_redirects=True)
        if r.status_code==200:
            self.content = str(r.content.decode()).split('\n')
        else:
            print("Erro: ", r.status_code)

    def novo_jogo(self, vidas=5):
        self.vidas = vidas
        self.palavra = random.choice(self.content)
        return len(self.palavra)

    def tentar_letra(self, letra):
        if self.vidas > 0:
            if letra in self.palavra:
                return [idx for idx in range(len(self.palavra)) if self.palavra[idx]==letra]
            else:
                self.vidas -= 1
                if self.vidas == 0:
                    print("Fim de jogo!")
                    return False
                else:
                    return []
        
    def tentar_palavra(self, palavra):
        if self.vidas > 0:
            if self.palavra == palavra:
                print ("Ganhou!")
                return True
            else:
                self.vidas = 0
                print("Fim de jogo!")
                return False

Criar Jogador automático 

In [ ]:
class JogadorAutomatico:
    def __init__(self):
        url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
        r = requests.get(url, allow_redirects=True)
        if r.status_code == 200:
            self.vocabulario = str(r.content.decode()).split('\n')
        else:
            raise Exception("Erro ao carregar o vocabulário")

    def calcular_frequencias(self, palavras):
        letras = Counter("".join(palavras))
        total = sum(letras.values())
        return {letra: count / total for letra, count in letras.items()}

    def calcular_entropia(self, palavras, letra):
        total = len(palavras)
        if total == 0:
            return 0
        contem_letra = sum([1 for palavra in palavras if letra in palavra])
        p_contem = contem_letra / total
        p_nao_contem = 1 - p_contem
        entropia = 0
        if p_contem > 0:
            entropia -= p_contem * log2(p_contem)
        if p_nao_contem > 0:
            entropia -= p_nao_contem * log2(p_nao_contem)
        return entropia

    def escolher_proxima_letra(self, palavras_candidatas, letras_escolhidas):
        frequencias = self.calcular_frequencias(palavras_candidatas)
        letras_validas = [letra for letra in frequencias if letra not in letras_escolhidas]
        entropias = {letra: self.calcular_entropia(palavras_candidatas, letra) for letra in letras_validas}
        return max(entropias, key=entropias.get)